# Chapter 6 Exercises
(questions are from 3rd edition from this chapter onward)

1. _What is the approximate depth of a Decision Tree trained (without restrictions) on a training set with one million instances?_<br>
<br>
O(log2(m)), so 6/log(2) = 19.931568569324174
<br>
<br>
1. _Is a node’s Gini impurity generally lower or greater than its parent’s? Is it generally lower/greater, or always lower/greater?_<br>
<br>
The CART algorithm stops if it cannot reduce impurity. So, if it is measuring the sum of the impurity of the children, weighted by the number of instances associated with the node, it might allow a node to have higher unweighted impurity. However, if it stops if any child node is not more pure than the parent (unweighted), then the children are guaranteed to be more pure.
<br><br>
1. _If a Decision Tree is overfitting the training set, is it a good idea to try decreasing max_depth?_<br>
<br>
yes
<br><br>
1. _If a Decision Tree is underfitting the training set, is it a good idea to try scaling the input features?_<br>
<br>
no, decision trees don't require scaling
<br><br>
1. _If it takes one hour to train a Decision Tree on a training set containing 1 million instances, roughly how much time will it take to train another Decision Tree on a training set containing 10 million instances? Hint: consider the CART algorithm’s computational complexity._<br>
<br>
training complexity is O(n x mlog2(m)),<br>
so 10^6 * log2(10^6) vs. 10^7 * log2(10^7)<br>
 19.931568569324174 vs. 10 * 23.253496664211536<br>
 11.666666666666666 times longer
<br><br>
1. _If it takes one hour to train a Decision Tree on a given training set, roughly how much time will it take if you double the number of features?_<br>
<br>
2 hours
<br><br>
1. _Train and fine-tune a Decision Tree for the moons dataset by following these steps:_<br>
 1. _Use make_moons(n_samples=10000, noise=0.4) to generate a moons dataset._<br>
 1. _Use train_test_split() to split the dataset into a training set and a test set._<br>
 1. _Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values for a DecisionTreeClassifier. Hint: try various values for max_leaf_nodes._<br>
 1. _Train it on the full training set using these hyperparameters, and measure your model’s performance on the test set. You should get roughly 85% to 87% accuracy._<br>

In [1]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X, y = make_moons(n_samples=10000, noise=0.4)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

param_grid = [
    {'max_leaf_nodes': [100, 1000, 10000], 'max_depth': [100,1000,10000]},
  ]

clf = DecisionTreeClassifier(random_state=42)

grid_search = GridSearchCV(clf, param_grid, cv=5, verbose=2)

main_pipeline = Pipeline([
    ('grid', grid_search)
])

pipe_out = main_pipeline.fit(X_train, y_train)
grid_search.best_params_

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END ..................max_depth=100, max_leaf_nodes=100; total time=   0.0s
[CV] END ..................max_depth=100, max_leaf_nodes=100; total time=   0.0s
[CV] END ..................max_depth=100, max_leaf_nodes=100; total time=   0.0s
[CV] END ..................max_depth=100, max_leaf_nodes=100; total time=   0.0s
[CV] END ..................max_depth=100, max_leaf_nodes=100; total time=   0.0s
[CV] END .................max_depth=100, max_leaf_nodes=1000; total time=   0.0s
[CV] END .................max_depth=100, max_leaf_nodes=1000; total time=   0.0s
[CV] END .................max_depth=100, max_leaf_nodes=1000; total time=   0.0s
[CV] END .................max_depth=100, max_leaf_nodes=1000; total time=   0.0s
[CV] END .................max_depth=100, max_leaf_nodes=1000; total time=   0.0s
[CV] END ................max_depth=100, max_leaf_nodes=10000; total time=   0.0s
[CV] END ................max_depth=100, max_leaf_

{'max_depth': 100, 'max_leaf_nodes': 100}

In [5]:
from sklearn.metrics import accuracy_score

y_predict = pipe_out.predict(X_train)
accuracy_score(y_train, y_predict)

0.886875

In [6]:
y_predict = pipe_out.predict(X_test)
accuracy_score(y_test, y_predict)

0.856

8. _Grow a forest by following these steps:_<br>

 1. _Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit-Learn’s ShuffleSplit class for this._<br>

 1. _Train one Decision Tree on each subset, using the best hyperparameter values found in the previous exercise. Evaluate these 1,000 Decision Trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely perform worse than the first Decision Tree, achieving only about 80% accuracy._<br>

 1. _Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision Trees, and keep only the most frequent prediction (you can use SciPy’s mode() function for this). This approach gives you majority-vote predictions over the test set._<br>

 1. _Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a Random Forest classifier!_<br>

In [19]:
from sklearn.model_selection import ShuffleSplit
import numpy as np
from scipy.stats import mode

N_SPLITS = 1000
ss = ShuffleSplit(n_splits=N_SPLITS, random_state=0, train_size=100)

y_predict100s = np.zeros((len(y_test), N_SPLITS))
accuracies = np.zeros(N_SPLITS)

for i, (train_index, test_index) in enumerate(ss.split(X_train, y_train)):
    clf = DecisionTreeClassifier(random_state=42, max_leaf_nodes=100, max_depth=100)
    clf.fit(X_train[train_index], y_train[train_index])
    p = clf.predict(X_test)
    for j, pr in enumerate(p):
        y_predict100s[j, i] = pr
    accuracies[i] = accuracy_score(y_test, p)
    
np.mean(accuracies)

0.7951785

In [21]:
forest_predict = np.zeros(len(y_test))
for i, z in enumerate(forest_predict):
    modes, counts = mode(y_predict100s[i])
    forest_predict[i] = modes[0]
    
accuracy_score(y_test, forest_predict)

0.869